In [ ]:
import ipyvuetify as v
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.display import display, clear_output
import json

### SQLytes Inventory Tracking

## Choose which requirement you want to view:

In [ ]:
backend = 'http://127.0.0.1:5000/sqlytes/'
# output_area = widgets.Output()

In [ ]:
class WarehouseWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses().get('Warehouses')
        self.current_warehouse_index = 0
        self.all_warehouses_df = pd.DataFrame()
       
        self.warehouses_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All parts for a warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )
        
        #self.warehouses_options_dropdown.observe(self.warehouse_options_dropdown_eventhandler, names='value')

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select wid:"
        )
        
        self.warehouse_selection.observe(self.on_selection_change)
        
        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        
    def getAvailableWarehouses(self):
        """Return all IDs of the warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available wid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return wids
    
    def getAllWarehouses(self):
        """Return all warehouses in the Data Base."""
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data

        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
    
    def displayNextWarehouse(self,_):
        self.current_warehouse_index = (self.current_warehouse_index+1) % len(self.warehouses)
        print(f"Displaying index {self.current_warehouse_index}")
        selected_warehouse = self.warehouses[self.current_warehouse_index]
        print(f"Displaying Warehouse ID: {selected_warehouse}")
        selected_warehouse_df = pd.DataFrame.from_dict(selected_warehouse, orient="index")
        self.all_warehouses_df = selected_warehouse_df
        with self.output_area:
            clear_output(wait=True)
            display(self.all_warehouses_df)
            
    def getWarehousePartsAPI(self,selected_warehouse):
        """Obtain all Parts for a warehouse.

        Args:
            wid (int): ID of the warehouse of which the parts are going to be showed.
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'warehouse/{selected_warehouse}/parts'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                all_parts_df = pd.DataFrame(data["WarehouseParts"])
                display(all_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse has no parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getWarehousePartsAPI(selected_warehouse)

In [ ]:
# warehouseWidgets = WarehouseWidgets()

In [ ]:
# warehouseWidgets.warehouses_options_dropdown.observe(warehouseWidgets.warehouse_options_handler,names='value')

In [ ]:
# Display the dropdown initially
#display(warehouseWidgets.warehouses_options_dropdown)

In [ ]:
def displayWarehouseWidgets():
    warehouseWidgets = WarehouseWidgets()
    warehouseWidgets.getWarehousePartsAPI(warehouseWidgets.warehouse_selection.value)
    warehouseWidgets.warehouse_selection.observe(warehouseWidgets.on_selection_change)
    display(warehouseWidgets.warehouse_selection)
    display(warehouseWidgets.output_area)

In [ ]:
# display(warehouseWidgets.warehouses_options_dropdown)

In [ ]:
# warehouseWidgets.displayNextWarehouse(_)

In [ ]:
# # Attach the function to the button's click event
# button = warehouseWidgets.next_warehouse_button
# button.on_click(warehouseWidgets.displayNextWarehouse)

# # Display the widgets
# display(warehouseWidgets.output_area)
# display(button)

In [ ]:
class SuppliedPartsWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_suppliers = self.getAvailableSuppliers()
        self.suppliers = self.getAllSuppliers()
        self.current_supplier_index = 0
        self.all_suppliers_df = pd.DataFrame()
       
        # self.suppliers_options_dropdown = widgets.Select(
        #     options=['All Suppliers info', 'All parts supplied by this supplier'],
        #     value='All Suppliers info',
        #     description='Search for:',
        #     rows=2,
        # )

        self.supplier_selection = widgets.Dropdown(
            options=self.available_suppliers,
            value=self.available_suppliers[0],
            description="Select sid:"
        )
        
        self.supplier_selection.observe(self.on_selection_change)
        
        # self.next_supplier_button = widgets.Button(
        #     description="Next Supplier",
        #     icon='check'
        # )
        
    def getAvailableSuppliers(self):
        """Return all IDs of the suppliers in the Data Base."""
        response = requests.get(f'{backend}{"supplier"}')
        # List to store all available sid's.
        sids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data:
                sids.append(record['id'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return sids
    
    def getAllSuppliers(self):
        """Return all suppliers in the Data Base."""
        response = requests.get(f'{backend}{"supplier"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data

        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
    
    # def displayNextSupplier(self,_):
    #     self.current_supplier_index = (self.current_supplier_index+1) % len(self.suppliers)
    #     print(f"Displaying index {self.current_supplier_index}")
    #     selected_supplier = self.suppliers[self.current_supplier_index]
    #     print(f"Displaying Warehouse ID: {selected_supplier}")
    #     selected_supplier_df = pd.DataFrame.from_dict(selected_supplier, orient="index")
    #     self.all_suppliers_df = selected_supplier_df
    #     with self.output_area:
    #         clear_output(wait=True)
    #         display(self.all_suppliers_df)
            
    def getSuppliedPartsAPI(self,selected_supplier):
        """Obtain all Parts for a warehouse.

        Args:
            wid (int): ID of the warehouse of which the parts are going to be showed.
            endpoint (str): Endpoint to be connected to the backend route to obtain the data.
        """
        endpoint = f'supplies/{selected_supplier}'
        response = requests.get(f'{backend}{endpoint}')
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                supplied_parts_df = pd.DataFrame(data)
                display(supplied_parts_df)
            elif response.status_code == 404:
                print('Error {}: Supplier does not supply any parts'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_supplier = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_supplier = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getSuppliedPartsAPI(selected_supplier)

In [ ]:
def displaySuppliedParts():
    suppliedPartsWidgets = SuppliedPartsWidgets()
    suppliedPartsWidgets.getSuppliedPartsAPI(suppliedPartsWidgets.supplier_selection.value)
    suppliedPartsWidgets.supplier_selection.observe(suppliedPartsWidgets.on_selection_change)
    display(suppliedPartsWidgets.supplier_selection)
    display(suppliedPartsWidgets.output_area)

In [ ]:
class TransactionWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.available_warehouses = self.getAvailableWarehouses()
        self.warehouses = self.getAllWarehouses()
        self.current_warehouse_index = 0
        self.all_transactions_df = pd.DataFrame()
       
        self.warehouse_options_dropdown = widgets.Select(
            options=['All Warehouses info', 'All transactions in this warehouse'],
            value='All Warehouses info',
            description='Search for:',
            rows=2,
        )

        self.warehouse_selection = widgets.Dropdown(
            options=self.available_warehouses,
            value=self.available_warehouses[0],
            description="Select wid:"
        )
        
        self.warehouse_selection.observe(self.on_selection_change)
        
        self.next_warehouse_button = widgets.Button(
            description="Next Warehouse",
            icon='check'
        )
        

    def getAvailableWarehouses(self):
        """
        Return all IDs of the warehouses in the DB
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # List to store all available sid's.
        wids = []
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            for record in data['Warehouses']:
                wids.append(record['wid'])
        else:
            print('Error {}: Could not get data from the backend.'.format(response.status_code))
        return sorted(wids)

    
    def getAllWarehouses(self):
        """
        Return all suppliers in the Data Base.
        """
        response = requests.get(f'{backend}{"warehouse"}')
        # Check if response was succesful.
        if response.status_code == 200:
            # Parse JSON Data from response.
            data = response.json()
            return data
        print(f'Error {response.status_code}: Could not get data from the backend.')
        return {}
                
    def getTransactionsPerWarehouseAPI(self, selected_warehouse):
        endpoint = f'transaction'
        response = requests.put(f'{backend}{endpoint}', data=json.dumps({"wid": selected_warehouse}), headers={"Content-Type": "application/json"})
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            if response.status_code == 200: 
                data = response.json()
                supplied_parts_df = pd.DataFrame(data["Result"])
                display(supplied_parts_df)
            elif response.status_code == 404:
                print('Error {}: Warehouse does not have any transactions'.format(response.status_code))
            else:
                print(f'{backend}{endpoint}')
                print('Error {}: Could not get data from the backend.'.format(response.status_code))
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_warehouse = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_warehouse    = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getTransactionsPerWarehouseAPI(selected_warehouse)


In [ ]:
def displayTransactionWidget():
    transactionWidget = TransactionWidgets()
    transactionWidget.getTransactionsPerWarehouseAPI(transactionWidget.warehouse_selection.value)
    transactionWidget.warehouse_selection.observe(transactionWidget.on_selection_change)
    display(transactionWidget.warehouse_selection)
    display(transactionWidget.output_area)

In [ ]:
class MenuWidgets:
    def __init__(self):
        self.output_area = widgets.Output()
        self.specs = ["Local Statistics", "Global Statistics", "Transactions in Warehouse", "Part Prices", "Parts Supplied by Suppliers", "Parts in Warehouse"]
        
        self.spec_selection = widgets.Dropdown(
            options=self.specs,
            value = self.specs[0],
            description="Select view:"
        )
        
        self.spec_selection.observe(self.on_selection_change)
        
    def getSpec(self,selected_spec):
        with self.output_area:
            clear_output(wait=True)  # Clear the previous output
            
            display_functions = {
                self.specs[2]: displayTransactionWidget,
                self.specs[4]: displaySuppliedParts,
                self.specs[5]: displayWarehouseWidgets,
            }
        
            display_function = display_functions.get(selected_spec, None)
        
            if display_function is not None:
                display_function()
            else:
                print(selected_spec)

                
            
                
    def on_selection_change(self,change):
        if isinstance(change, int):
            # Directly received the selected value, not an event dictionary
            selected_supplier = change
        elif change['type'] == 'change' and change['name'] == 'value':
            # Received an event dictionary, extract the selected value
            selected_supplier = change['new']
        else:
            # Invalid change, do nothing
            return
        self.getSpec(selected_supplier)

In [ ]:
menuWidget = MenuWidgets()
menuWidget.getSpec(menuWidget.spec_selection.value)
menuWidget.spec_selection.observe(menuWidget.on_selection_change)
display(menuWidget.spec_selection)
display(menuWidget.output_area)